# 06_saving_consecutive_metrics_all_models
* Calculating the consecutive metrics for all models and then saving this to a folder with in the consecutve metrics folder with the folder name being the window length.

PHD-10
* Newer more streamlined version from Documents/PhD/03_SN_global_different_windows_(nb25).ipynb
* ALl S/N calculation and bound are handled by function

# Preamble

In [1]:
import numpy as np
import os, sys
from time import perf_counter
import itertools
import matplotlib.pyplot as plt
import xarray as xr
from importlib import reload
from glob import glob
import cartopy.crs as ccrs
from multiprocessing import Pool
from typing import Dict, List
from warnings import filterwarnings
filterwarnings('ignore')
from functools import partial
import constants

sys.path.append(constants.MODULE_DIR)
import xarray_class_accessors as xca
import signal_to_noise
import open_ds
from classes import ExperimentTypes
import utils
from classes import LongRunMIPError
logger = utils.get_notebook_logger()

In [2]:
def open_all_needed_netcdf_files(control_fpath: str, experiment_fpath: str, mask=None):
    '''
    Open all needed experiments
    '''
    open_kwargs = dict(requested_length=100, mask=mask)
    print(f'{open_kwargs=}')
    ds_experiment = open_ds.read_longrunmip_netcdf(experiment_fpath, ROOT_DIR_REGRID, **open_kwargs)
    ds_control = open_ds.read_longrunmip_netcdf(control_fpath, ROOT_DIR_REGRID, **open_kwargs)
    ds_experiment_lowess = open_ds.read_longrunmip_netcdf(experiment_fpath, ROOT_DIR_LOWESS, **open_kwargs)
    
    return ds_experiment.to_array(), ds_control.to_array(), ds_experiment_lowess.to_array()

In [3]:
def open_calculate_sn_then_save(model:str, window: int, mask:str=None, debug=False):
    '''
    * Open the control, experiment and lowess experiment
    * Calculate singal to noise
    * Save data
    '''
    print(f'\n ==== {model} \n')
    control_fpath = [fname for fname in files_to_open_control if model in fname.lower()][0]
    experiment_fpath = [fname for fname in files_to_open_experiment if model in fname.lower()][0]
    
    ds_experiment, ds_control, ds_experiment_lowess =\
            open_all_needed_netcdf_files(control_fpath, experiment_fpath, mask)
    print(f'- {model}| All files have been opened')

    
    # For testing: .isel(lat=slice(0,2), lon=slice(0,2), time=slice(0,60))
    sn_ds = signal_to_noise.calculate_multi_window_rolling_signal_to_nosie_and_bounds(
        ds_experiment, ds_control, 
        lowess_experiment_da = ds_experiment_lowess, start_window = window,
    name=model, logginglevel='ERROR')
    if debug:
        return sn_ds
    save_name = os.path.join(OUTPUT_DIR_2, f'{model}_signal_to_noise.nc')
    print(f'{model} -> {save_name}')
    sn_ds.to_netcdf(save_name)

In [14]:
variable = 'pr' # tas, tos, pr

In [16]:
MASK_MAP = {'tas': None, 'tos': 'sea', 'sic': 'sea', 'pr': None}
mask = MASK_MAP[variable]
mask

In [17]:
chunks = {'lat':48/2,'lon':96/2,'time':-1}
ROOT_DIR = os.path.join(constants.LONGRUNMIP_DIR, variable)
ROOT_DIR_REGRID = os.path.join(ROOT_DIR, 'regrid_retimestamped')
ROOT_DIR_LOWESS = os.path.join(ROOT_DIR, 'lowess')
print(ROOT_DIR_REGRID, ROOT_DIR_LOWESS, sep='\n')

/g/data/w40/ab2313/PhD/longrunmip/pr/regrid_retimestamped
/g/data/w40/ab2313/PhD/longrunmip/pr/lowess


In [18]:
models_to_get = open_ds.get_models_longer_than_length()
utils.pprint_list(models_to_get, num_start_items=6)

lenght = 7
 0. ccsm3
 1. cesm104
 2. cnrmcm61
 3. hadcm3l
 4. ipslcm5a
 5. mpiesm11


In [19]:
files_to_open_experiment = open_ds.get_file_names_from_from_directory(ROOT_DIR_REGRID,
                                                           ExperimentTypes.ABRUPT4X,
                                                           models_to_get)

files_to_open_control = open_ds.get_file_names_from_from_directory(ROOT_DIR_REGRID,
                                                           ExperimentTypes.CONTROL,
                                                           models_to_get)
utils.pprint_list(files_to_open_experiment)

lenght = 7
 0. pr_mon_CCSM3_abrupt4x_2120_g025.nc
 1. pr_mon_CESM104_abrupt4x_5900_g025.nc


In [20]:
models = [fname.split('_')[2].lower() for fname  in files_to_open_experiment]#constants.LONGRUNMIP_MODELS#
models

['ccsm3', 'cesm104', 'cnrmcm61', 'hadcm3l', 'ipslcm5a', 'mpiesm11', 'mpiesm12']

In [21]:
constants.WINDOWS_OF_INTEREST

[20, 80, 150]

In [22]:
for window in constants.WINDOWS_OF_INTEREST:
    
    print(window)

    OUTPUT_DIR = os.path.join(ROOT_DIR, 'signal_to_noise')
    OUTPUT_DIR_2 = os.path.join(OUTPUT_DIR, f'window_{window}')

    print(OUTPUT_DIR, OUTPUT_DIR_2, sep='\n')
    utils.mkdir_no_error(OUTPUT_DIR)
    utils.mkdir_no_error(OUTPUT_DIR_2)
    

    with Pool() as pool:
        pool.map(partial(open_calculate_sn_then_save, window=window, mask=mask), models)

20
/g/data/w40/ab2313/PhD/longrunmip/pr/signal_to_noise
/g/data/w40/ab2313/PhD/longrunmip/pr/signal_to_noise/window_20

 ==== ipslcm5a 

 ==== hadcm3l 

 ==== cesm104 

 ==== cnrmcm61 

 ==== mpiesm12 

 ==== ccsm3 

 ==== mpiesm11 







open_kwargs={'requested_length': 100, 'mask': None}open_kwargs={'requested_length': 100, 'mask': None}open_kwargs={'requested_length': 100, 'mask': None}open_kwargs={'requested_length': 100, 'mask': None}open_kwargs={'requested_length': 100, 'mask': None}open_kwargs={'requested_length': 100, 'mask': None}
open_kwargs={'requested_length': 100, 'mask': None}





- Removing first 10 steps
- cesm104| All files have been opened

Experiment
--------

[20]
20, - hadcm3l| All files have been opened

Experiment
--------

[20]
20, - ipslcm5a| All files have been opened

Experiment
--------

[20]
20, - ccsm3| All files have been opened

Experiment
--------

[20]- mpiesm12| All files have been opened


Experiment
--------
20, 
[20]
20, - mpiesm11| All files hav

In [12]:
# reload(signal_to_noise)
# for model in models:
    
#     t1 = perf_counter()
#     print(model, end = '')
    
#     control_fpath = [fname for fname in files_to_open_control if model in fname.lower()][0]
#     experiment_fpath = [fname for fname in files_to_open_experiment if model in fname.lower()][0]
    
#     try:
#         ds_experiment, ds_control, ds_experiment_lowess =\
#                     open_all_needed_netcdf_files(control_fpath, experiment_fpath)
#         print('| All files have been opened')
        
        
#         sn_ds = signal_to_noise.calculate_multi_window_rolling_signal_to_nosie_and_bounds(
#             ds_experiment, ds_control,
#             lowess_experiment_da = ds_experiment_lowess, start_window = 21)
        
#         sn_ds.to_netcdf(os.path.join(OUTPUT_DIR_2, f'{model}_signal_to_noise.nc'))

#         t2 = perf_counter()
#         print(f' - {(t2-t1)/60}')
        
#     except LongRunMIPError as e:
#         logger.error(e)
    

In [13]:

#         ds_sn, ds_sn_stable, ds_sn_increasing, ds_sn_decreasing =\
#                 signal_to_noise.global_sn(
#             ds_experiment.isel(lat=slice(0,5), lon=slice(0,4), time=slice(0,40)),
#             ds_control.isel(lat=slice(0,5), lon=slice(0,4), time=slice(0,40)), 
#             ds_experiment_lowess.isel(lat=slice(0,5), lon=slice(0,4), time=slice(0,40)),
#             window=RUN_PARAMETERS['window'])

#         save_all_netcdf_files(ds_sn, ds_sn_stable, ds_sn_increasing, ds_sn_decreasing)